# Convertible Bond Valuation and Convergence Analysis

In [36]:
# This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
# without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  
# See the license for more details.

Value a Convertible Bond and compare to blog report at http://gouthamanbalaraman.com/blog/value-convertible-bond-quantlib-python.html

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from financepy.finutils.FinDate import FinDate
from financepy.products.bonds.FinBondConvertible import FinBondConvertible
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinCalendar import *
from financepy.market.curves.FinFlatCurve import FinFlatCurve
from financepy.finutils.FinSchedule import FinSchedule

## Creating the Convertible Bond

In [4]:
valuationDate = FinDate(9, 1, 2004)
settlementDate = valuationDate.addDays(2)

We set the maturity date, annualised coupon, coupon frequency and convention for calculating accrued interest

In [27]:
maturityDate = FinDate(15, 3, 2022)
coupon = 0.0575
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
accrualBasis = FinDayCountTypes.THIRTY_360_BOND
face = 100.0 
conversionRatio = 3.84615  

We need to set the face amount and the conversion ratio. This is the number of shares you can convert to for that face amount.

We can also delay the start of the conversion feature using the start convert date. We allow conversion to occur from settlement.

In [28]:
startConvertDate = settlementDate

We can also set the call schedule - the price and date on which the issuer can call back the bond. This is given with reference to a price of 100. Making the call price very high means it will never be called. So the call has no impact.

In [29]:
callPrice = 100.0
callDates = [FinDate(20, 3, 2007), FinDate(15, 3, 2012), FinDate(15, 3, 2017)]
callPrices = [callPrice, callPrice, callPrice]

We can also set the put schedule - the price and date on which the owner can put back the bond to the issuer. This is given with reference to a price of 100. Making the put price very low means it will never be put.

In [30]:
putPrice = 100.0
putDates = [FinDate(20, 3, 2007), FinDate(15, 3, 2012), FinDate(15, 3, 2017)]
putPrices = [putPrice, putPrice, putPrice]

At this point we can create our bond.

In [31]:
bond = FinBondConvertible(maturityDate, coupon, frequencyType, startConvertDate, conversionRatio,
                          callDates, callPrices, putDates, putPrices, accrualBasis, face)

In [32]:
print(bond)

MATURITY DATE: TUE 15 MAR 2022
COUPON: 0.0575
FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: FinDayCountTypes.THIRTY_360_BOND
FACE AMOUNT: 100.0
CONVERSION RATIO: 3.84615
START CONVERT DATE: SUN 11 JAN 2004
CALL: DATES
TUE 20 MAR 2007: 100.0
THU 15 MAR 2012: 100.0
WED 15 MAR 2017: 100.0
PUT: DATES
TUE 20 MAR 2007: 100.0
THU 15 MAR 2012: 100.0
WED 15 MAR 2017: 100.0



## Valuation Inputs

We need to specify the valuation date

The model allows a discrete dividend schedule. We can use the FinSchedule class to do this.

In [33]:
dividendSchedule = FinSchedule(settlementDate, maturityDate).flows()[1:]
dividendYields = [0.02] * len(dividendSchedule)

In [34]:
stockPrice = 29.04
stockVolatility = 0.40
rate = 0.04
discountCurve = FinFlatCurve(settlementDate, rate, 1)
creditSpread = 0.03
recoveryRate = 0.40 # 40 percent
numStepsPerYear = 500

In [35]:
bond.value(settlementDate, stockPrice, stockVolatility, dividendSchedule, dividendYields,
           discountCurve, creditSpread, recoveryRate, numStepsPerYear)

{'cbprice': 135.7842542364674,
 'bond': 90.15036818623973,
 'delta': 2.8665124338108035,
 'gamma': 0.40855954000551076,
 'theta': 159.73734692064627}

Copyright (c) 2020 Dominic O'Kane

This compares to 132.31 found by QL. The difference could be due to the different treatment of dividends or the credit spread.